In [15]:
# Dependencies
import json
import requests
from configg import api_key
from california_cities import ca_cities
import pyowm
import pandas as pd


In [16]:
#ca_city_list = ca_cities()
city_df = pd.read_csv("california_cities.csv")
#city_df.head()
city_df.head()

,Unnamed: 0,city,type,county,state,lat,lng
0,2,Adelanto,City,San Bernardino,california,34.582770,-117.409215
1,3,Agoura Hills,City,Los Angeles,california,34.153340,-118.761676
2,4,Alameda,City,Alameda,california,37.765206,-122.241636
3,5,Albany,City,Alameda,california,37.886870,-122.297748
4,6,Alhambra,City,Los Angeles,california,34.095287,-118.127015


In [17]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"
city = "Los Angeles"


owm = pyowm.OWM(api_key) 

#with open('dataWeather.json','w') as outfile:
# Build query URL
#    for city in ca_city_list:
#        query_url = url + "appid=" + api_key + "&q=" + city
#        weather_response = requests.get(query_url)
#        weather_json = weather_response.json()
#        json.dump(weather_json, outfile)
 #       print(f"The weather API responded with: {weather_json}.")
# Get the temperature from the response
#observation = owm.weather_around_coords(40.71, -74.00)
observation = owm.weather_at_coords(-0.107331,51.503614) 
w = observation.get_weather()
print(w.get_temperature('fahrenheit')['temp'])
#w.get_wind()['speed']
#w.get_clouds() 

city_df['Date']=w.get_reference_time(timeformat='iso')

81.67


In [18]:
city_df.head()

,Unnamed: 0,city,type,county,state,lat,lng,Date
0,2,Adelanto,City,San Bernardino,california,34.582770,-117.409215,2018-12-01 21:59:45+00
1,3,Agoura Hills,City,Los Angeles,california,34.153340,-118.761676,2018-12-01 21:59:45+00
2,4,Alameda,City,Alameda,california,37.765206,-122.241636,2018-12-01 21:59:45+00
3,5,Albany,City,Alameda,california,37.886870,-122.297748,2018-12-01 21:59:45+00
4,6,Alhambra,City,Los Angeles,california,34.095287,-118.127015,2018-12-01 21:59:45+00


In [19]:
#temperature add
weather_data = []
for i, r in city_df.iterrows():
    weather = owm.weather_at_coords(r.loc['lat'], r.loc['lng'])
    weather_list = weather.get_weather()
    weather_data.append(weather_list.get_temperature('fahrenheit')['temp'])

weather = pd.Series(weather_data)
city_df['Temperature'] = weather.values

#,obervation.get_weather().get_humidity(),obervation.get_weather().get_temperature('celsius') 

In [20]:
final_df = city_df.drop(['Unnamed: 0'],axis=1)
final_df.to_csv("weather_temp.csv")

In [21]:
#get cloud_data
cloud_data = []
for i, r in city_df.iterrows():
    cloud = owm.weather_at_coords(r.loc['lat'], r.loc['lng'])
    cloud_list = cloud.get_weather()
    cloud_data.append(cloud_list.get_clouds())

cloud_df = pd.Series(cloud_data)
city_df['Cloud'] = cloud_df.values


In [22]:
city_final_df = city_df.drop(['Unnamed: 0'],axis=1)
city_final_df.to_csv("weather_cloud.csv")
city_final_df.head()

,city,type,county,state,lat,lng,Date,Temperature,Cloud
0,Adelanto,City,San Bernardino,california,34.582770,-117.409215,2018-12-01 21:59:45+00,57.13,40
1,Agoura Hills,City,Los Angeles,california,34.153340,-118.761676,2018-12-01 21:59:45+00,63.01,1
2,Alameda,City,Alameda,california,37.765206,-122.241636,2018-12-01 21:59:45+00,57.88,90
3,Albany,City,Alameda,california,37.886870,-122.297748,2018-12-01 21:59:45+00,58.35,40
4,Alhambra,City,Los Angeles,california,34.095287,-118.127015,2018-12-01 21:59:45+00,63.27,1


In [23]:
#get pressure data
pressure_data = []
for i, r in city_df.iterrows():
    pressure = owm.weather_at_coords(r.loc['lat'], r.loc['lng'])
    pressure_list = pressure.get_weather()
    pressure_data.append(pressure_list.get_pressure()['press'])

pressure_df = pd.Series(pressure_data)
city_df['Pressure'] = pressure_df.values

In [24]:
city_df.to_csv("weather_all.csv")
city_df.head()

,Unnamed: 0,city,type,county,state,lat,lng,Date,Temperature,Cloud,Pressure
0,2,Adelanto,City,San Bernardino,california,34.582770,-117.409215,2018-12-01 21:59:45+00,57.13,40,1012.0
1,3,Agoura Hills,City,Los Angeles,california,34.153340,-118.761676,2018-12-01 21:59:45+00,63.01,1,1013.0
2,4,Alameda,City,Alameda,california,37.765206,-122.241636,2018-12-01 21:59:45+00,57.88,90,1015.0
3,5,Albany,City,Alameda,california,37.886870,-122.297748,2018-12-01 21:59:45+00,58.35,40,1015.0
4,6,Alhambra,City,Los Angeles,california,34.095287,-118.127015,2018-12-01 21:59:45+00,63.27,1,1013.0


In [25]:
#get wind data
wind_speed_data = []
wind_gust_data = []
for i, r in city_df.iterrows():
    wind = owm.weather_at_coords(r.loc['lat'], r.loc['lng'])
    wind_list = wind.get_weather()
    wind_speed_data.append(wind_list.get_wind()['speed'])
    #wind_gust_data.append(wind_list.get_wind()['gust'])

wind_df = pd.Series(wind_speed_data)
city_df['Wind'] = wind_df.values
#wind_gust_df = pd.Series(wind_gust_data)
#city_df['Wind(gust)'] = wind_gust_df.values


In [26]:
city_df = city_df.drop(['Unnamed: 0'],axis=1)
#city_final_df = city_df.drop(['Wind'],axis=1)
city_df.to_csv("weather_final.csv")
city_df.head()

,city,type,county,state,lat,lng,Date,Temperature,Cloud,Pressure,Wind
0,Adelanto,City,San Bernardino,california,34.582770,-117.409215,2018-12-01 21:59:45+00,57.13,40,1012.0,7.7
1,Agoura Hills,City,Los Angeles,california,34.153340,-118.761676,2018-12-01 21:59:45+00,63.01,1,1013.0,4.1
2,Alameda,City,Alameda,california,37.765206,-122.241636,2018-12-01 21:59:45+00,57.88,90,1015.0,6.7
3,Albany,City,Alameda,california,37.886870,-122.297748,2018-12-01 21:59:45+00,58.35,40,1015.0,6.7
4,Alhambra,City,Los Angeles,california,34.095287,-118.127015,2018-12-01 21:59:45+00,63.27,1,1013.0,4.1
